In [75]:
#import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,precision_score, recall_score, f1_score



In [76]:
df_train=pd.read_csv('df_train.csv')
df_test=pd.read_csv('df_test.csv')


In [77]:
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report

# Define the target columns
target_cols = ['IS_HCP', 'TAXONOMY']

# Extract the features (X) and labels (y) from the DataFrame
X = df_train.drop(target_cols, axis=1)
y = df_train[target_cols]


In [78]:
common_columns = set(X.columns).intersection(df_test.columns)
X = X[common_columns]
test_data = df_test[common_columns]

In [79]:
#split the data into train and test
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

X_train.shape

(91149, 1430)

In [86]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

models = {
    # 'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    # 'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'Naive Bayes': GaussianNB()
}


In [87]:

result_df = pd.DataFrame(columns=['Model', 'Precision', 'F1 Score', 'Recall', 'Accuracy'])
for model_name, model in models.items():
    multioutput_model = MultiOutputClassifier(model)
    multioutput_model.fit(X_train, y_train)
    y_pred = multioutput_model.predict(X_test)
    # precision = precision_score(y_test, y_pred, average='macro')
    # recall = recall_score(y_test, y_pred, average='macro')
    # f1 = f1_score(y_test, y_pred, average='macro')
    
# Predict on the test data

    # Evaluate the performance
    accuracy = multioutput_model.score(X_test, y_test)
    print(model_name, accuracy)



Decision Tree 0.8806389327716342
Random Forest 0.8979287344216255
AdaBoost 0.7269176759698087
Naive Bayes 0.027207302088818677


In [88]:
#best model
model=RandomForestClassifier()
model.fit(X_train,y_train)


RandomForestClassifier()

In [89]:

y_pred=model.predict(test_data)

In [90]:
y_pred

array([[  0, 149],
       [  1, 149],
       [  0, 149],
       ...,
       [  0, 149],
       [  0, 149],
       [  1, 149]], dtype=int64)

In [108]:
y_pred_df = pd.DataFrame(y_pred, columns=[target_cols[0],target_cols[1]])


In [109]:

y_pred_df.head()

,IS_HCP,TAXONOMY
0,0,149
1,1,149
2,0,149
3,0,149
4,1,143


In [110]:
result_df = pd.concat([df_test['PK'], y_pred_df], axis=1)
result_df = result_df[['PK', 'IS_HCP', 'TAXONOMY']]


In [111]:
df=pd.read_csv('Doceree-HCP_Train.csv')

from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
le.fit_transform(df['TAXONOMY'])
result_df['TAXONOMY']=le.inverse_transform(y_pred[:,1])

In [112]:
result_df.head()

,PK,IS_HCP,TAXONOMY
0,2,0,2084P0800X
1,3,1,2084P0800X
2,4,0,2084P0800X
3,5,0,2084P0800X
4,6,1,2084N0400X


In [113]:
result_df['TAXONOMY'].value_counts()

2084P0800X    25601
2084N0400X     1557
207Q00000X      449
207R00000X      265
2085R0001X      122
              ...  
2080P0203X        1
207QA0505X        1
2084N0600X        1
163WP0808X        1
2084A2900X        1
Name: TAXONOMY, Length: 73, dtype: int64

In [114]:
result_df.to_csv('submission.csv',index=False)